In [ ]:
# import os
# import json
# import pandas as pd
# import numpy as np
# import base64
# import requests
# import asyncio
# import aiohttp
# import nest_asyncio
# import time

# nest_asyncio.apply()
# global all_data_results
# base_directory = "/Users/muhammadarbabarshad/Downloads/Plant_Phenotyping_Datasets/Plant_Phenotyping_Datasets/Plant/Ara2013-Canon"
# all_data = pd.read_csv(base_directory + "/Leaf_counts.csv", header=None)

# def load_image(image_path: str) -> str:
#     """Load image from file and encode it as base64."""
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# api_key = os.getenv("OPENAI_API_KEY")
# model = "gpt-4-turbo"
# url = "https://api.openai.com/v1/chat/completions"
# headers = {
#     "Content-Type": "application/json",
#     "Authorization": f"Bearer {api_key}"
# }

# vision_prompt = """
# Given the image, provide the number of leaves in the picture. Few-shot examples (with image and corresponding outputs) may (or may not) be provided to you. You can learn from those and then predict for the current given image. Be careful, the leafs are convoluted and hard to see so they might look less than anticipated. Give answer in the following JSON format:
# {"leaf_count": number_of_leaves}
# """

# max_requests_per_minute = 80
# request_times = []

# async def get_image_informations(session, inputs: dict) -> dict:
#     """Invoke model with image and prompt."""
#     global request_times
#     current_time = time.time()
    
#     # Remove old request times outside the current time window
#     while request_times and request_times[0] <= current_time - 60:
#         request_times.pop(0)
    
#     if len(request_times) >= max_requests_per_minute:
#         # If the rate limit is exceeded, wait until the oldest request is outside the window
#         time_to_wait = request_times[0] + 60 - current_time
#         await asyncio.sleep(time_to_wait)
    
#     request_times.append(current_time)
    
#     payload = {
#         "model": model,
#         "messages": [
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": vision_prompt},
#                     *inputs['examples'],
#                     {
#                         "type": "image_url",
#                         "image_url": {
#                             "url": f"data:image/jpeg;base64,{inputs['image']}",
#                             "detail": "high"
#                         }
#                     }
#                 ]
#             }
#         ],
#         "max_tokens": 4096
#     }
#     async with session.post(url, headers=headers, json=payload) as response:
#         result = await response.json()
#         print(result)
#         if "choices" in result and result["choices"]:
#             return result
#         else:
#             raise Exception("Unexpected API response format")

# async def process_image(session, i, number_of_shots, all_data_results):
#     try:
#         image_path = base_directory + "/" + all_data[0][i] + "_rgb.png"
#         image_base64 = load_image(image_path)

#         examples = []
#         for j in range(max(0, i - number_of_shots), i):
#             example_image_path = base_directory + "/" + all_data[0][j] + "_rgb.png"
#             example_image_base64 = load_image(example_image_path)
#             examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
#             examples.append({"type": "text", "text": f"Leaf count: {all_data.at[j, 1]}"})

#         result = await get_image_informations(session, {"image": image_base64, "examples": examples})
#         leaf_count = result["choices"][0]['message']['content']
#         all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(leaf_count)['leaf_count']
#     except Exception as e:
#         print(f"Error processing {all_data[0][i]}_rgb.png: {str(e)}")
#         all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'

# async def process_images_for_shots(number_of_shots, all_data_results):
#     async with aiohttp.ClientSession() as session:
#         tasks = []
#         for i in range(10, min(9999999, len(all_data))):
#             task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
#             tasks.append(task)
#         await asyncio.gather(*tasks)

# async def main():
#     all_data_results = all_data.copy(deep=True)
#     all_data_results.columns = all_data_results.columns.map(str)

#     for number_of_shots in [0, 1, 2, 4, 8]:
#         await process_images_for_shots(number_of_shots, all_data_results)
#     all_data_results.to_csv("current_all_data_results_Ara2013-Canon.csv")
#     # print(all_data_results.iloc[9:16])
#     print("done")

# asyncio.run(main())

# modifying for fruits

In [4]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/fruits-images-dataset-object-detection/Train File/Train File'
# FOR API
max_requests_per_minute = 10

num_instances_per_class=10

# Define the list of expected class labels
expected_classes = ['Banana', 'Jackfruit', 'Mango', 'Litchi', 'Hog Plum', 'Papaya', 'Grapes', 'Apple', 'Orange', 'Guava']

# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all files in the directory
for filename in os.listdir(base_directory):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        # Extract the label from the filename
        label = filename.split('_')[1].split('.')[0]
        
        # Find the closest match among expected classes using fuzzy matching
        closest_match = process.extractOne(label, expected_classes)
        
        # Get the standardized label (closest match)
        standardized_label = closest_match[0]
        
        # Append the file path and standardized label to the respective lists
        file_paths.append(os.path.join(base_directory, filename))
        labels.append(standardized_label)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)


def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class of the fruit presented. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    current_time = time.time()
    
    # Remove old request times outside the current time window
    while request_times and request_times[0] <= current_time - 60:
        request_times.pop(0)
    
    if len(request_times) >= max_requests_per_minute:
        # If the rate limit is exceeded, wait until the oldest request is outside the window
        time_to_wait = request_times[0] + 60 - current_time
        await asyncio.sleep(time_to_wait)
    
    request_times.append(current_time)
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("01-Fruit.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

{'id': 'chatcmpl-9Stoyn0TW4lhY5qFjIPF56yNUGUqf', 'object': 'chat.completion', 'created': 1716674788, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"prediction": "Hog Plum"}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 382, 'completion_tokens': 8, 'total_tokens': 390}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9Stoy4EhKIPdNdXAuivthCpa9yRRm', 'object': 'chat.completion', 'created': 1716674788, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "Litchi"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 552, 'completion_tokens': 11, 'total_tokens': 563}, 'system_fingerprint': 'fp_927397958d'}
{'id': 'chatcmpl-9StoyCjvi9734PUE1YuUUfFRN15Vs', 'object': 'chat.completion', 'created': 1716674788, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "pr

In [9]:
all_data_results= pd.read_csv("01-Fruit.csv")
all_data_results.head()
#calculate wrong predictions
wrong_predictions = all_data_results[all_data_results['1'] != all_data_results['# of Shots 0']]
print(f"Number of wrong predictions: {len(wrong_predictions)}")

Number of wrong predictions: 2


In [ ]:
import os;os.getenv("OPENAI_API_KEY")